Santander Dev Week 2023 (ETL com Python)

In [1]:
import json

# --- Nome do arquivo JSON local com os 5 usuários ---
LOCAL_MOCK_FILE = 'all_users_mock.json'

users = []
try:
    with open(LOCAL_MOCK_FILE, 'r', encoding='utf-8') as f:
        # Carrega a lista completa de usuários (5 objetos) do arquivo local
        users = json.load(f)

    print(f"✅ Extração de {len(users)} usuários do arquivo local concluída.")

except FileNotFoundError:
    print(f"❌ ERRO: Arquivo {LOCAL_MOCK_FILE} não encontrado. Certifique-se de que ele foi enviado ao Colab.")

# Exibe o JSON extraído para confirmação
print(json.dumps(users, indent=2))

✅ Extração de 5 usuários do arquivo local concluída.
[
  {
    "id": 1,
    "name": "Clara Mendes",
    "account": {
      "id": 9999,
      "number": "01.097587-1",
      "agency": "1009",
      "balance": 1000.0,
      "limit": 500.0
    },
    "card": {
      "id": 8888,
      "number": "4722513401777777",
      "limit": 100.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 2,
    "name": "Lucas Oliveira",
    "account": {
      "id": 9999,
      "number": "01.097587-1",
      "agency": "1009",
      "balance": 1000.0,
      "limit": 500.0
    },
    "card": {
      "id": 8888,
      "number": "4722513401777777",
      "limit": 100.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 3,
    "name": "Beatriz Neves",
    "account": {
      "id": 9999,
      "number": "01.097587-1",
      "agency": "1009",
      "balance": 1000.0,
      "limit": 500.0
    },
    "card": {
      "id": 8888,
      "number": "4722513401777777",
      "limit": 100.0
    },
    "featu

Extract

In [2]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1, 2, 3, 4, 5]


In [4]:
# Instala a biblioteca do Google Gemini (google-genai) e requests (usada na Extração/Carga).
!pip install google-genai requests

Transform

In [5]:
import os
from google import genai
from google.genai import types

# Documentação Oficial da API Gemini: https://ai.google.dev/
# Para gerar uma API Key GRÁTIS do Gemini:
# 1. Acesse o Google AI Studio
# 2. Clique em "Create API Key"
# Link direto: https://ai.google.dev/gemini-api/docs/api-key

# ⚠️ SUBSTITUA O 'TODO' pela sua chave de API GRATUITA do Google Gemini.
GEMINI_API_KEY = 'TODO'

# Inicializa o cliente da API.
try:
    client = genai.Client(api_key=GEMINI_API_KEY)
    print("Cliente Gemini inicializado com sucesso.")
except Exception as e:
    # Este bloco garante que o notebook não trave se a chave não for válida.
    print(f"ERRO: Não foi possível inicializar o cliente Gemini. Verifique a chave 'GEMINI_API_KEY'.")
    client = None

Cliente Gemini inicializado com sucesso.


In [6]:
from google import genai
from google.genai import types

def generate_ai_news(user):
  """
  Utiliza a API do Gemini 2.5 Flash para gerar a mensagem de marketing.
  """
  if 'client' not in globals() or client is None:
      return f"Falha na IA: Cliente Gemini não inicializado. Verifique a chave."

  # 1. Definição da persona (System Instruction no Gemini)
  system_instruction = "Você é um especialista em markting bancário, criando mensagens persuasivas, úteis e motivadoras sobre investimentos."

  # 2. Conteúdo da requisição (Prompt)
  prompt = f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"

  # 3. Estrutura e chamada da API do Gemini
  try:
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        # *** LINHA CORRIGIDA AQUI ***
        # Usando o construtor explícito 'types.Part(text=prompt)'
        contents=[types.Content(parts=[types.Part(text=prompt)])],
        # ****************************
        config=types.GenerateContentConfig(
            system_instruction=system_instruction
        )
    )
    return response.text.strip()
  except Exception as e:
      return f"Erro ao chamar a API Gemini: {e}"


# Loop para processar os usuários e adicionar as notícias (Lógica original mantida):
for user in users:
  news = generate_ai_news(user)
  print(f"Mensagem para {user['name']}: {news}")

  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Mensagem para Clara Mendes: Clara, investir é crucial para seu futuro e sonhos. Faça seu dinheiro trabalhar por você. Comece hoje!
Mensagem para Lucas Oliveira: Lucas, invista agora para um futuro mais tranquilo e próspero. Faça seu dinheiro trabalhar por você!
Mensagem para Beatriz Neves: Beatriz, seu futuro merece! Invista hoje e construa a vida que você sonha. ✨
Mensagem para Thiago Costa: Thiago, seus sonhos merecem mais. Invista e transforme planos em realidade!
Mensagem para Juliana Melo: Juliana, invista no seu futuro! Faça seu dinheiro crescer e realize seus sonhos. Comece hoje!


In [7]:
import requests

sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app' # Endereço original

def update_user(user):
  # Mesmo que falhe, o código demonstra a intenção e a função Load.
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!") # Isso provavelmente imprimirá 'False!'

User Clara Mendes updated? False!
User Lucas Oliveira updated? False!
User Beatriz Neves updated? False!
User Thiago Costa updated? False!
User Juliana Melo updated? False!


Relatório

In [8]:
import json
import os

# Nome do arquivo de relatório
report_filename = 'relatorio_final_gemini.json'

# --- 1. Salvar o JSON Completo (Resultado Final) ---
try:
    with open(report_filename, 'w', encoding='utf-8') as f:
        # Usa json.dump para escrever a lista completa 'users' no arquivo
        json.dump(users, f, indent=4, ensure_ascii=False)

    print(f"\n✅ Relatório JSON Completo salvo em: {report_filename}")
    print("Você pode fazer o download deste arquivo no seu ambiente Colab.")

except Exception as e:
    print(f"❌ Erro ao salvar o arquivo JSON: {e}")


# --- 2. Exibir um resumo das mensagens geradas ---
print("\n--- RESUMO DAS MENSAGENS GERADAS ---")
for user in users:
    # A última notícia na lista 'news' é a que o Gemini acabou de gerar
    last_news_index = len(user['news']) - 1
    new_message = user['news'][last_news_index]['description']

    print(f"Usuário ID {user['id']} ({user['name']}):")
    print(f"  Mensagem IA: '{new_message}'")

print("-------------------------------------")


✅ Relatório JSON Completo salvo em: relatorio_final_gemini.json
Você pode fazer o download deste arquivo no seu ambiente Colab.

--- RESUMO DAS MENSAGENS GERADAS ---
Usuário ID 1 (Clara Mendes):
  Mensagem IA: 'Clara, investir é crucial para seu futuro e sonhos. Faça seu dinheiro trabalhar por você. Comece hoje!'
Usuário ID 2 (Lucas Oliveira):
  Mensagem IA: 'Lucas, invista agora para um futuro mais tranquilo e próspero. Faça seu dinheiro trabalhar por você!'
Usuário ID 3 (Beatriz Neves):
  Mensagem IA: 'Beatriz, seu futuro merece! Invista hoje e construa a vida que você sonha. ✨'
Usuário ID 4 (Thiago Costa):
  Mensagem IA: 'Thiago, seus sonhos merecem mais. Invista e transforme planos em realidade!'
Usuário ID 5 (Juliana Melo):
  Mensagem IA: 'Juliana, invista no seu futuro! Faça seu dinheiro crescer e realize seus sonhos. Comece hoje!'
-------------------------------------
